In [1]:
import Pkg

In [2]:
Pkg.activate(".")

Activating environment at `~/DistancePlan/Project.toml`


In [3]:
using Revise

┌ Info: Precompiling Revise [295af30f-e4ad-537b-8983-00126c2a3abe]
└ @ Base loading.jl:1273


In [4]:
using StaticArrays

In [9]:
includet("src/DistancePlan.jl")

In [12]:
s1 = DistancePlan.Sphere([1. 1.], 2.)

Main.DistancePlan.Sphere([1.0 1.0], 2.0)

In [13]:
s2 = DistancePlan.Sphere([3. 1.], 1.)

Main.DistancePlan.Sphere([3.0 1.0], 1.0)

In [14]:
d = DistancePlan.norm(s2.center - s1.center)

2.0

In [ ]:
x = 2 * s1.radius ^ 2 - s2.radius ^ 2 / (2 * s1.radius)

In [17]:
function cone_angle(sphere_a, sphere_b)
    assert(isapprox(sum((sphere_a.center - sphere_b.center) ^ 2), sphere_a.radius ^2))
    # "Align" the spheres along the x axis, and take an arbitrary orthogonal y axis.
    # Then we would have the following equations:
    #   x^2 + y^2 = R^2
    #   (x - R)^2 + y^2 = r^2
    # Solving for y^2:
    #   x^2 + y^2 = R^2         =>   y^2 = R^2 - x^2
    #   (x - R)^2 + y^2 = r^2   =>   y^2 = r^2 - (x - R)^2
    # Combining the equations:
    #   R^2 - x^2 = r^2 - (x - R)^2
    #   R^2 - x^2 - r^2 + (x^2 - 2xR + R^2) = 0
    #   -x^2 + x^2 - 2Rx + 2R^2 - r^2 = 0
    #   -2Rx = -2R^2 + r^2
    #   x = (2R^2 - r^2) / 2R
    x = (2 * sphere_a.radius ^ 2 - sphere_b.radius^2) / (2 * sphere_a.radius)
    # Now we need the angle, so we use cos(x / R):
    return cos(x / sphere_a.radius)
end

cone_angle (generic function with 1 method)

In [ ]:
function lens_center(sphere_a, sphere_b)
    offset = sphere_a.center - sphere_b.center
    d = sqrt(sum(offset .^ 2))
    # "Align" the spheres along the x axis, and take an arbitrary orthogonal y axis.
    # Then we would have the following equations:
    #   x^2 + y^2 = R^2
    #   (x - d)^2 + y^2 = r^2
    # Solving for y^2:
    #   x^2 + y^2 = R^2         =>   y^2 = R^2 - x^2
    #   (x - d)^2 + y^2 = r^2   =>   y^2 = r^2 - (x - d)^2
    # Combining the equations:
    #   R^2 - x^2 = r^2 - (x - R)^2
    #   R^2 - x^2 - r^2 + (x^2 - 2xd + d^2) = 0
    #   -x^2 + x^2 - 2dx + d^2 + R^2 - r^2 = 0
    #   -2dx = -2R^2 + r^2
    #   x = (d^2 + R^2 - r^2) / 2d
    x = (d ^ 2 + sphere_a.radius ^ 2 - sphere_b.radius^2) / (2 * d)
    return sphere_a.center +. offset * (x / d)
end

In [18]:
using LinearAlgebra

In [20]:
struct Arm
    lengths
    joint_lows
    joint_highs
end

In [38]:
Vector(s1.center[1]:.2:s2.center[1])

11-element Array{Float64,1}:
 1.0
 1.2
 1.4
 1.6
 1.8
 2.0
 2.2
 2.4
 2.6
 2.8
 3.0

In [43]:
function query_spheres(joint_locations, obstacles, points_per_link=3)
    Channel() do channel
        for i in 1:(length(joint_locations) - 1)
            start = joint_locations[i]
            stop = joint_locations[i + 1]
            for point in LinRange(start, stop, points_per_link)
                d = DistancePlan.distance(point, obstacles)
                put!(channel, (i, DistancePlan.Sphere(point, d)))
            end
        end
    end
end

query_spheres (generic function with 2 methods)

In [49]:
for (link_number, sphere) in query_spheres([[0., 0.], [1., 1.], [2., 1.]], [[.5, .6]]
    println(link_number, ": ", sphere)
end

1: Main.DistancePlan.Sphere([0.0, 0.0], 0.7810249675906654)
1: Main.DistancePlan.Sphere([0.5, 0.5], 0.09999999999999998)
1: Main.DistancePlan.Sphere([1.0, 1.0], 0.6403124237432849)
2: Main.DistancePlan.Sphere([1.0, 1.0], 0.6403124237432849)
2: Main.DistancePlan.Sphere([1.5, 1.0], 1.077032961426901)
2: Main.DistancePlan.Sphere([2.0, 1.0], 1.5524174696260025)


In [42]:
LinRange(s1.center, s2.center, 3)

3-element LinRange{Array{Float64,2}}:
 [1.0 1.0],[2.0 1.0],[3.0 1.0]

Generally, the kinematic volume planning algorithm extension step works as follows:

For each linkage, sample distances along that linkage, producing a sequence of hyper-spheres.
For each pair of adjacent hyper-spheres in the sequence, compute the largest hyper-sphere which overlaps both of them, by finding the hyper-sphere one dimension lower. These overlap hyper-spheres are called the "safe spheres."
For each joint, compute the angle that joint can move in each safe sphere after it in the kinematic chain. Take the smallest of these angles for that joint. That is the joints "safe angle."

Save all joints patch angles (as well as the central angle) as a patch into the patch tree.

When executing motion within the patch, the sum of angles divided by their respective safe angles should be less than one. This produces a diagonal patch in the angle space.

The reason this works is that change in angle corresponds to change in the distance of the linkage to the safe circle. Specifically, it corresponds to a portion of the distance equal the ratio of the angle over the safe angle. Alternatively, this can be seen as adding the fractions of the arcs together which the different joints would sweep out. Since the arcs aren't overlapping (assuming linkages have non-zero length), the length of their arc lengths in sequence will always be greater than the distance between their joined endpoints (i.e. the triangle inequality). However, this bound is not tight. For example, linkages at orthogonal angles can get arbitrarily close to their full individual ranges of motion. Unfortunately, performing projections on a shape that tightly bounds the safe angle space corresponding to the safe spheres would be quite difficult, since every such shape would be different. This bound can be arbitrarily close to tight, if the two joints are close to each other, and the second linkage is very long.

In [ ]:
struct PatchTree
  centers::Vector
  patch_sizes::Vector
  children::Vector{Vector{Int}}
  parent::Vector{Int}
  PatchTree() = new([], [], [], [])
end